In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()
import pyspark 
import warnings
from pyspark.sql import SQLContext

sc = pyspark.SparkContext()
sqlContext = SQLContext(sc)

# **Training**




In [ ]:
Data_set = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('/content/drive/Shareddrives/YT-data/clean_tweet.csv')
Data_set = Data_set.dropna()
(train_set, val_set, test_set) = Data_set.randomSplit([0.98, 0.01, 0.01], seed = 2000)


In [ ]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

from pyspark.ml.classification import LogisticRegression

from pyspark.ml.evaluation import BinaryClassificationEvaluator

from pyspark.ml.feature import  VectorAssembler


In [ ]:

tokenizer = Tokenizer(inputCol="text", outputCol="tokens")

C_V = CountVectorizer(vocabSize=2**16, inputCol="tokens", outputCol='cv')

idf = IDF(inputCol='cv', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms

indexer = StringIndexer(inputCol = "target", outputCol = "label")

lr = LogisticRegression()

IDF_pipeline = Pipeline(stages=[tokenizer, A_V, idf, indexer, lr])

In [ ]:

IDF_pipelineFit = IDF_pipeline.fit(train_set)
IDF_predictions = IDF_pipelineFit.transform(val_set)



In [ ]:
train_accuracy = IDF_predictions.filter(IDF_predictions.label == IDF_predictions.prediction).count() / float(IDF_predictions.count())

print(train_accuracy)


# Testing

In [ ]:
IDF_test_predictions = IDF_pipelineFit.transform(test_set)
test_accuracy = IDF_test_predictions.filter(IDF_test_predictions.label == IDF_test_predictions.prediction).count() / float(test_set.count())

# print accuracy, roc_auc
print(test_accuracy)


# Level of engagement 
### positivity

In [ ]:
API_comment = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('/content/drive/Shareddrives/YT-data/API_data.csv')

data = API_comment.selectExpr( "video_id as video_id","comment_text as text")
from pyspark.sql.functions import col, concat_ws
data = data.withColumn("text", concat_ws(",",col("text")))




### DATA CLEANING

In [ ]:
from nltk.tokenize import WordPunctTokenizer
from bs4 import BeautifulSoup
import re

toknizer = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def data_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped_text = soup.get_text()
    stripped = re.sub(combined_pat, '', souped_text)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters = re.sub("[^a-zA-Z]", " ", clean)
    lowercase = letters.lower()
    words = toknizer.tokenize(lowercase)
    return (" ".join(words)).strip()

In [ ]:
 
data = data.rdd.map(lambda x : (x[0] , data_cleaner(x[1])) ).toDF().selectExpr( "_1 as video_id","_2 as text")


In [ ]:
test_predictions = IDF_pipelineFit.transform(data.limit(250000))

In [ ]:
out = test_predictions[["video_id" , "prediction"]]


In [ ]:
count = out.rdd.map(lambda x : (x[0] , 1)).reduceByKey(lambda x ,y : x+y)
psitive = out.rdd.map(lambda x : (x[0] , x[1])).reduceByKey(lambda x ,y : x+y)
positive_count= psitive.join(count)
positivity = positive_count.map(lambda x : (x[0] , x[1][0] / x[1][1]))

Avg. likes for positivity

In [ ]:
API_like_dislike_data = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('/content/drive/Shareddrives/YT-data/API_stat.csv')

ld = API_like_dislike_data.selectExpr( "video_id as video_id","likeCount as likes" ,"dislikeCount as dislikes" , "viewCount as view")


In [ ]:
likes_num = ld.rdd.map(lambda x : (x[0] , x[1]))


In [ ]:
psitivity_reacts = positivity.join(likes_num)
count_pos = psitivity_reacts.map(lambda x: (round(x[1][0],1) ,1)).reduceByKey(lambda x  ,y : x+y)
commulative_likes = psitivity_reacts.map(lambda x: (x[1][0] ,x[1][1])).reduceByKey(lambda x  ,y : x+y)
likes_count= count_pos.join(commulative_likes)
pos_avgreacts = likes_count.map(lambda x : (x[0] , x[1][1] / x[1][0]))


In [ ]:
sc.stop()